In [8]:
import pandas as pd
import duckdb
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

In [4]:
df = pd.read_csv('01_datasets/01_calfire_incidents.csv')
df.to_parquet('01_datasets/01_calfire_incidents.parquet', engine="pyarrow",index=False)

print("Parquet file created")

Parquet file created


In [7]:
con = duckdb.connect(database=":memory:")

con.execute("CREATE TABLE incidents AS SELECT * FROM parquet_scan('01_datasets/01_calfire_incidents.parquet')")

result = con.execute("SELECT * FROM incidents LIMIT 5").fetchdf()
print(result)


   incident_name incident_is_final incident_date_last_update  \
0    Bridge Fire                 Y      2018-01-09T13:46:00Z   
1      Pala Fire                 Y      2020-09-16T14:07:35Z   
2     River Fire                 Y      2022-10-24T11:39:23Z   
3  Fawnskin Fire                 Y      2013-04-22T09:00:00Z   
4      Gold Fire                 Y      2013-05-01T07:00:00Z   

  incident_date_created          incident_administrative_unit  \
0  2017-10-31T11:22:00Z       Shasta-Trinity National Forest    
1  2009-05-24T14:56:00Z               CAL FIRE San Diego Unit   
2  2013-02-24T08:16:00Z          CAL FIRE San Bernardino Unit   
3  2013-04-20T17:30:00Z        San Bernardino National Forest   
4  2013-04-30T12:59:00Z  CAL FIRE Madera-Mariposa-Merced Unit   

   incident_administrative_unit_url incident_county  \
0                               NaN          Shasta   
1                               NaN       San Diego   
2                               NaN            Inyo   
3   

In [10]:
def convert_to_parquet():
    df = pd.read_csv("01_datasets/01_calfire_incidents.csv")
    df.to_parquet("01_datasets/01_calfire_incidents.parquet", engine="pyarrow", index=False)

default_args = {
    'owner': 'airflow',
    'start_date': datetime(2025, 2, 1),
}

dag = DAG('incident_data_pipeline', default_args=default_args, schedule_interval='@daily')

task = PythonOperator(
    task_id='convert_csv_to_parquet',
    python_callable=convert_to_parquet,
    dag=dag
)

/var/folders/p7/m480s0s56rb3pl4d11ss4wch0000gn/T/ipykernel_86449/1538362978.py:10 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.